In [ ]:
import pandas as pd
dir_hackathon = '.'
dir_data_in = f'{dir_hackathon}/Data/In'
dir_data_out = f'{dir_hackathon}/Data/Out'

Загрузка данных. Колонки "Коды ТН ВЭД ЕАЭС", "Технические регламенты" и "Группа продукции" - категориальные. Нам нужно их обработать и превратить в численные. Применим два подхода - нумерацию и разбиение на много колонок.

In [ ]:
all_data_in = pd.read_excel(f'{dir_data_in}/dataset2.xlsx', header=0)
all_data_in.columns = ['Номер продукции', 'Коды ТН ВЭД ЕАЭС', 'Технические регламенты', 'Группа продукции', 'Общее наименование продукции', 
                       'ИЛ', 'Заявитель', 'Адрес Заявителя', 'Изготовитель', 'Страна', 'Адрес изготовителя']

all_data_in.head(2)

Нам нужно обработать  колонки 'Коды ТН ВЭД ЕАЭС', 'Технические регламенты', 'Группа продукции как категориальные.  У нас есть категориальные данные двух видов. Группы продукции - самое простое, просто строка, обозначающее название группы. Обработаем штатными средствами. С двумя другими сложнее - там в каждой ячейке группы перечислены через разделитель, да ещё и с дубликатами. Их будем обрабатывать отдельно. 

Nan - это пустые строчки, где не заданы значения. Заменим на "Not presented". 

In [ ]:
all_data_in['Коды ТН ВЭД ЕАЭС'] = all_data_in['Коды ТН ВЭД ЕАЭС'].fillna(value="Not presented") # почему-то inplace не работает
all_data_in['Коды ТН ВЭД ЕАЭС'] = all_data_in['Коды ТН ВЭД ЕАЭС'].astype(str)
all_data_in['Коды ТН ВЭД ЕАЭС'].tail(5) #там были NaN


In [ ]:
#предобработка столбца с кодами ВЭД ЕАЭС чтобы убрать повторы
def remove_replies_from_celll(cell, delimiter=";"):
    splitted_cell = [s.strip() for s in cell.split(delimiter)]
    unique_splitted_cell = list(set(splitted_cell))
    result_cell = delimiter.join(unique_splitted_cell)
    return result_cell

all_data_in['Коды ТН ВЭД ЕАЭС'] = all_data_in['Коды ТН ВЭД ЕАЭС'].apply(lambda x: remove_replies_from_celll(x))
all_data_in.head(5)


In [ ]:
object_cols = ['Группа продукции', 'Технические регламенты', 'Коды ТН ВЭД ЕАЭС']
object_cols_target = ['Группа продукции_ordinal', 'Технические регламенты_ordinal', 'Коды ТН ВЭД ЕАЭС_ordinal'] # В такую колонку положим результат простой нумерации
all_data_in[object_cols]


Посмотрим, сколько есть различных категорий

In [ ]:
for col in object_cols:
    count = len(all_data_in[col].value_counts())
    print(f"{col} : {count}")

Ordinal encoder. Нумеруем категории. 


In [ ]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
data_copy = all_data_in.copy()
all_data_in[object_cols_target] = ordinal_encoder.fit_transform(data_copy[object_cols])
all_data_in[object_cols + object_cols_target].head(10)

Сохраним новый датасет

In [ ]:
all_data_in.to_excel(f'{dir_data_out}/dateset_categorical.xlsx')